## Lock Analysis Example

In [3]:
import transmap
import folium
import pandas as pd
import requests
import datetime
import pandas as pd
from urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

make_url = lambda uri: "https://oak.cast.uark.edu/{0}".format(uri)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
metadata = transmap.services.cwbi.locks.metadata()

start_latitude = 37.0902
start_longitude = -95.7129

folMap = folium.Map(zoom_start=4,min_zoom=4,max_zoom=14, location=[start_latitude, start_longitude])
for lock in metadata:
    if lock.longitude and lock.latitude:
        marker = folium.Marker(
            location = [lock.latitude, lock.longitude],
            tooltip = f"{lock.rivername} - {lock.lockno}",
            popup = lock.lockname
        )
        marker.add_to(folMap)
folMap

In [9]:
traffic_data = []

for lock in metadata:
    try:
        traffic = transmap.services.lpms.locks.lock_traffic(
            river_code = lock.rivercode, 
            lock_number = lock.lockno
        )
        traffic_data.append(traffic)
    except:
        continue

In [8]:
ais_data = requests.get(make_url('ais/vessels'), params={
    "limit": 10000,
    "start": 0,
    "start_date": "2019-05-29T00:00:00",
    "end_date": "2019-05-30T00:00:00",
}).json()